# Foundations of Computer Science Project

## Import packages

Tutti i pacchetti utilizzati per il progetto.

In [1]:
import pandas as pd
import numpy as np
import re
import time

Con time decido di monitorare il tempo di esecuzione dell'intero codice.

In [2]:
start = time.time()

## Import datasets

Import dei dataset ed ispezione.

In [3]:
data_ps = pd.read_csv('googleplaystore.csv')
data_ur = pd.read_csv('googleplaystore_user_reviews.csv')

In [4]:
data_ps.head(2) #Check the googleplaystore dataset

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up


In [5]:
data_ur.head(2) #Check the googleplaystore_user_reviews dataset

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462


Durante l'esercizio 2 mi sono reso conto che il record 10472 possiede una codifica errata di tutti gli attributi dovuta ad un problema verificatisi durante la raccolta dei dati. Ho proceduto quindi all'**eliminazione** di tale record.

In [6]:
data_ps.loc[[10472]]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [7]:
data_ps = data_ps.drop(10472)

## 1. Convert the app sizes to a number

Per evitare che 'Varies with device' crei problemi lo converto ad un forma da cui posso tornare indietro.

In [8]:
data_ps['Size'] = [re.sub('Varies with device', '99999999', size) for size in data_ps['Size']] 
data_ps['Size'] = [re.sub(',', '.', size) for size in data_ps['Size']] #Format 

In [9]:
sizes_to_num_re = re.compile('(?P<amount>\d*\.*\d*)(?P<unit>\w*\+*)') #

def unit_to_mult(unit):
    if unit == 'G':
        return 1000000000
    if unit == 'M':
        return 1000000 #Identifying the number of bytes into 1 Gb, Mb and Kb
    if unit == 'k':
        return 1000
    return 1

def to_numeric(elem): #Converting the strings into numbers
    m = sizes_to_num_re.search(elem)
    unit = m.group('unit')
    mult = unit_to_mult(unit)
    amount = float(m.group('amount'))
    return int(amount * mult)

In [10]:
data_ps['ExtSize'] = data_ps['Size'].apply(to_numeric) #Applying the function

Dopo aver convertito Size riporto 'Varies with device' alla sua forma originale per preservarne il significato.

In [11]:
data_ps['Size'] = [re.sub('99999999', 'Varies with device', size) for size in data_ps['Size']] #Reconverting the value '99999999' into 'Varies with device'
data_ps['ExtSize'] = data_ps['ExtSize'].apply(str)
data_ps['ExtSize'] = [re.sub('99999999', 'Varies with device', size) for size in data_ps['ExtSize']]
data_ps[['App','Size','ExtSize']].head()

,App,Size,ExtSize
0,Photo Editor & Candy Camera & Grid & ScrapBook,19M,19000000
1,Coloring book moana,14M,14000000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",8.7M,8700000
3,Sketch - Draw & Paint,25M,25000000
4,Pixel Draw - Number Art Coloring Book,2.8M,2800000


## 2. Convert the number of installs to a number

In [12]:
data_ps.dtypes #Check the dataframe's columns dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
ExtSize            object
dtype: object

In [13]:
data_ps.groupby('Installs')['Installs'].size() #Check for some strange values

Installs
0                    1
0+                  14
1+                  67
1,000+             907
1,000,000+        1579
1,000,000,000+      58
10+                386
10,000+           1054
10,000,000+       1252
100+               719
100,000+          1169
100,000,000+       409
5+                  82
5,000+             477
5,000,000+         752
50+                205
50,000+            479
50,000,000+        289
500+               330
500,000+           539
500,000,000+        72
Name: Installs, dtype: int64

In [14]:
data_ps['NumInstalls'] = [re.sub('\\D', '', number) for number in data_ps['Installs']] #Rimuovo tutti i caratteri
data_ps['NumInstalls'] = data_ps['NumInstalls'].apply(int) #Converting to int
data_ps[['App','Installs','NumInstalls']].head()

,App,Installs,NumInstalls
0,Photo Editor & Candy Camera & Grid & ScrapBook,"10,000+",10000
1,Coloring book moana,"500,000+",500000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...","5,000,000+",5000000
3,Sketch - Draw & Paint,"50,000,000+",50000000
4,Pixel Draw - Number Art Coloring Book,"100,000+",100000


In [15]:
data_ps['NumInstalls'].dtypes

dtype('int64')

## 3. Transform “Varies with device” into a missing value

In [16]:
data_psnan = data_ps.replace('Varies with device', np.nan)

Siccome a questo punto mi sono liberato definitivamente di 'Varies with device' posso convertire 'ExtSize' in float.

In [17]:
data_psnan['ExtSize'] = data_psnan['ExtSize'].apply(float) #Converting into float
data_psnan['ExtSize'].dtypes #Checking

dtype('float64')

## 4. Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

In [18]:
data_psnan[['App','Current Ver', 'Android Ver']].head()

,App,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,1.0.0,4.0.3 and up
1,Coloring book moana,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,NaN,4.2 and up
4,Pixel Draw - Number Art Coloring Book,1.1,4.4 and up


Sia per Current Version che per Android Version rimuovo tutti i caratteri che non sono numeri.

In [19]:
data_psnan['CurrVer'] = data_psnan['Current Ver']
data_psnan['AndroVer'] = data_psnan['Android Ver']
data_psnan['CurrVer'].replace('[a-z]+|[A-z]+','', regex = True, inplace = True)
#Remove everything that is not a number from Current Ver
data_psnan['AndroVer'].replace('[a-z]+|[A-z]+','', regex = True, inplace = True)
#Remove everything that is not a number from Android Ver
data_psnan['CurrVer'].replace(r'(\.+$)','', regex = True, inplace = True )
data_psnan['AndroVer'].replace(r'(\.+$)','', regex = True, inplace = True )
#Rimossi anche eventuali punti alla fine delle versioni
data_psnan[['App','Android Ver','Current Ver','AndroVer','CurrVer']].head() #Did it

,App,Android Ver,Current Ver,AndroVer,CurrVer
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.0.3 and up,1.0.0,4.0.3,1.0.0
1,Coloring book moana,4.0.3 and up,2.0.0,4.0.3,2.0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.0.3 and up,1.2.4,4.0.3,1.2.4
3,Sketch - Draw & Paint,4.2 and up,NaN,4.2,NaN
4,Pixel Draw - Number Art Coloring Book,4.4 and up,1.1,4.4,1.1


## 5. Remove the duplicates

Per affrontare il problema dei duplicati alla luce del fatto che si tratta di duplicati dovuti ad una raccolta di dati in periodi temporalmente diversi ho deciso di utilizzare le Reviews come discriminante per decidere quale sia il duplicato più recente e, quindi, quello contenente le informazioni di maggior interesse.

In [20]:
len(data_psnan)

10840

In [21]:
data_psnan.drop_duplicates(inplace = True) #Eliminiamo i duplicati identici tra di loro
len(data_psnan) #Qualche duplicato integrale c'era.

10357

In [22]:
data_psnan["Reviews"] = pd.to_numeric(data_psnan["Reviews"]) #Utilizziamo il numero di reviews come discriminante per capire quale sia il duplicato più recente
data_psnan[data_psnan['App'] == 'Facebook'][['App','Reviews']] #Prediamo questa applicazione come riferimento

,App,Reviews
2544,Facebook,78158306
3943,Facebook,78128208


Una volta deciso di affrontare il problema dei duplicati attraverso la discriminante delle Reviews ho ritenuto sufficiente ordinare il dataset in modo descrescente proprio in base a questa variabile imponendo poi l'eliminazione dei duplicati preservando i primi e, quindi, quelli con il numero di Reviews maggiore.

In [23]:
data_psnan = data_psnan.sort_values('Reviews', ascending = False) 
#Ordiniamo per il numero di reviews
data_psnan[data_psnan['App'] == 'Facebook'][['App','Reviews']] #Andato a buon fine

,App,Reviews
2544,Facebook,78158306
3943,Facebook,78128208


In [24]:
data_psnan.drop_duplicates(subset = 'App', keep = 'first', inplace = True) #Eliminiamo i duplicati che differiscono per il numero di reviews mantenendo solo quelli com le reviews più alte
data_psnan[data_psnan['App'] == 'Facebook'][['App','Reviews']] #E infatti è andato a buon fine

,App,Reviews
2544,Facebook,78158306


In [25]:
len(data_psnan) #Did it

9659

## 6. For each category, compute the number of apps

In [26]:
data_psnan.groupby('Category').size().sort_values(ascending = False)

Category
FAMILY                 1878
GAME                    945
TOOLS                   829
BUSINESS                420
MEDICAL                 395
PERSONALIZATION         376
PRODUCTIVITY            374
LIFESTYLE               369
FINANCE                 345
SPORTS                  325
COMMUNICATION           315
HEALTH_AND_FITNESS      288
PHOTOGRAPHY             281
NEWS_AND_MAGAZINES      254
SOCIAL                  239
BOOKS_AND_REFERENCE     222
TRAVEL_AND_LOCAL        219
SHOPPING                202
DATING                  170
VIDEO_PLAYERS           164
MAPS_AND_NAVIGATION     131
FOOD_AND_DRINK          112
EDUCATION               105
ENTERTAINMENT            86
AUTO_AND_VEHICLES        85
LIBRARIES_AND_DEMO       84
WEATHER                  79
HOUSE_AND_HOME           73
EVENTS                   64
ART_AND_DESIGN           61
PARENTING                60
COMICS                   56
BEAUTY                   53
dtype: int64

## 7. For each category, compute the average rating

In [27]:
data_psnan.groupby('Category')[['Rating']].mean().sort_values(by = 'Rating', ascending = False).head(10)

,Rating
Category,
EVENTS,4.435556
ART_AND_DESIGN,4.359322
EDUCATION,4.349038
BOOKS_AND_REFERENCE,4.344970
PERSONALIZATION,4.332215
PARENTING,4.300000
BEAUTY,4.278571
SOCIAL,4.247291
GAME,4.244432


## 8. Create two dataframes: one for the genres and one bridging apps and genders. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

Per affrontare questo problema ho deciso, per quanto riguarda la creazione del dataframe contenente i generi, di splittare banalmente laddove le tuple contengano array di generi utilizzando come discriminante il simbolo ";".

In [28]:
data_psnan['GenresList'] = [genre.split(';') for genre in data_psnan['Genres']]
#Splitto gli array di generi
genres = data_psnan.GenresList.apply(pd.Series) #genres dataframe
genres = genres.rename(columns = {0:'First_genre', 1:'Second_genre'})
genres.head(10)
#Così ottengo il dataframe dei generi

,First_genre,Second_genre
2544,Social,NaN
336,Communication,NaN
2604,Social,NaN
382,Communication,NaN
1879,Strategy,NaN
4005,Tools,NaN
1872,Arcade,NaN
3665,Video Players & Editors,NaN
7536,Tools,NaN
1878,Strategy,NaN


Una volta creato il dataframe dei generi è stato sufficiente, una volta creato un dataframe contenente i nomi delle app, unirli facendo riferimento ai rispettivi indici, così da ottenere per ogni app tante ripetizioni quanti fossero i generi a loro assegnati, realizzando quindi a tutti gli effetti una tabella ponte.

In [29]:
app = data_psnan['App']
#Bridgind table
genres_1 = pd.merge(genres, app.to_frame(), left_index = True, right_index = True)\
           .melt(id_vars = ['App'], value_name = "Single_genre")\
           .drop("variable", axis = 1)\
           .dropna() #per evitare che le app non aventi un secondo genere si ripresentino.
genres_1.head(10)
#Così ho realizzato la tabella ponte

,App,Single_genre
0,Facebook,Social
1,WhatsApp Messenger,Communication
2,Instagram,Social
3,Messenger – Text and Video Chat for Free,Communication
4,Clash of Clans,Strategy
5,Clean Master- Space Cleaner & Antivirus,Tools
6,Subway Surfers,Arcade
7,YouTube,Video Players & Editors
8,"Security Master - Antivirus, VPN, AppLock, Boo...",Tools
9,Clash Royale,Strategy


In [30]:
genres_1[genres_1['App'] == 'Pixel Draw - Number Art Coloring Book'] #Did it

,App,Single_genre
4831,Pixel Draw - Number Art Coloring Book,Art & Design
14490,Pixel Draw - Number Art Coloring Book,Creativity


## 9. For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

Per effettuare questo compito è stato sufficiente esplicitare una funzione ad hoc per assolvere al task. Come riferimento per i generi è stata utilizzata la colonna GenresList, creata nel punto precedente, con i generi presi singolarmente, così da preservare la distinzione tra i generi esplicitata nel punto precedente.

In [31]:
def create_dummies( df, colname ):
    col_dummies = pd.get_dummies(df[colname].apply(pd.Series).stack()).sum(level=0)
    col_dummies.drop(col_dummies.columns[0], axis=1, inplace=True)
    df = pd.concat([df, col_dummies.astype(bool)], axis=1)
    df.drop(colname, axis = 1, inplace = True )
    return df

In [32]:
genres_2 = create_dummies(data_psnan, 'GenresList')
genres_2[genres_2['App'] == 'Coloring book moana'][['App','Genres', 'Art & Design', 'Pretend Play', 'Creativity']] #Un esempio

,App,Genres,Art & Design,Pretend Play,Creativity
2033,Coloring book moana,Art & Design;Pretend Play,True,True,False


## 10. For each genre, compute the average rating. What is the genre with highest average?

Anche per questo compito, al fine di preservare i generi distinti e di non utilizzare la codifica Genres originale, ho proceduto ad unire il dataframe originale con il dataframe con i generi distinti così da poter calcolare il rating medio per ogni singolo genere.

In [33]:
genres_3 = genres_1.merge(data_psnan, on = 'App')
genres_3[['App','Genres','Single_genre']].head()

,App,Genres,Single_genre
0,Facebook,Social,Social
1,WhatsApp Messenger,Communication,Communication
2,Instagram,Social,Social
3,Messenger – Text and Video Chat for Free,Communication,Communication
4,Clash of Clans,Strategy,Strategy


In [34]:
genres_3.groupby('Single_genre')[['Rating']].mean().sort_values(by = 'Rating', ascending = False).head(10)

,Rating
Single_genre,
Events,4.435556
Puzzle,4.370732
Brain Games,4.358065
Art & Design,4.350000
Books & Reference,4.343275
Personalization,4.332215
Word,4.318182
Creativity,4.306250
Music & Audio,4.300000


In [35]:
genres_3.groupby('Single_genre')['Rating'].mean().idxmax()

'Events'

In [36]:
genres_3.groupby('Single_genre')['Rating'].mean().max()

4.435555555555556

## 11. For each app, compute the approximate income, obtain as a product of number of installs and price.

Banalmente per questo compito ho proceduto ad **eliminare** tutti i caratteri speciali da Price e a **convertirlo** in un attributo numerico per poi procedere al calcolo dei guadagni approssimativi.

In [37]:
data_psnan['NumPrice'] = [re.sub('\$','', price) for price in data_psnan['Price']]
data_psnan['NumPrice'] = data_psnan['NumPrice'].apply(float)
data_psnan['Income'] = data_psnan['NumPrice'] * data_psnan['NumInstalls']
data_psnan[['App', 'Income']].sort_values(by = 'Income', ascending = False).head(10)

,App,Income
2241,Minecraft,69900000.0
5351,I am rich,39999000.0
5356,I Am Rich Premium,19999500.0
4034,Hitman Sniper,9900000.0
7417,Grand Theft Auto: San Andreas,6990000.0
2883,Facetune - For Free,5990000.0
5578,Sleep as Android Unlock,5990000.0
8804,DraStic DS Emulator,4990000.0
4367,I'm Rich - Trump Edition,4000000.0
5354,I am Rich Plus,3999900.0


## 12. For each app, compute its minimum and maximum Sentiment_polarity

Per questo compito ho proceduto ad individuare per ogni App la **massima** e la **minima** sentiment_polarity singolarmente per poi concludere creando un dataframe che sintetizzi entrambi gli aspetti.

In [38]:
data_ur[['App', 'Sentiment_Polarity']].head(10)

,App,Sentiment_Polarity
0,10 Best Foods for You,1.00
1,10 Best Foods for You,0.25
2,10 Best Foods for You,NaN
3,10 Best Foods for You,0.40
4,10 Best Foods for You,1.00
5,10 Best Foods for You,1.00
6,10 Best Foods for You,0.60
7,10 Best Foods for You,NaN
8,10 Best Foods for You,0.00
9,10 Best Foods for You,0.00


In [39]:
data_max = data_ur.groupby('App')['Sentiment_Polarity'].max()
data_max = pd.DataFrame(data_max)
data_max = data_max.rename(columns = {'Sentiment_Polarity':'Max_Sentiment_Polarity'})
data_max.head()

,Max_Sentiment_Polarity
App,
10 Best Foods for You,1.000000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000
11st,1.000000
1800 Contacts - Lens Store,0.838542
1LINE – One Line with One Touch,1.000000


In [40]:
data_min = data_ur.groupby('App')['Sentiment_Polarity'].min()
data_min = pd.DataFrame(data_min)
data_min = data_min.rename(columns = {'Sentiment_Polarity':'Min_Sentiment_Polarity'})
data_min.head()

,Min_Sentiment_Polarity
App,
10 Best Foods for You,-0.8000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,-0.1125
11st,-1.0000
1800 Contacts - Lens Store,-0.3000
1LINE – One Line with One Touch,-0.8250


In [41]:
data_sentiment_polarity = data_max.merge(data_min, left_index = True, right_index = True)
data_sentiment_polarity.head(10)

,Max_Sentiment_Polarity,Min_Sentiment_Polarity
App,,
10 Best Foods for You,1.000000,-0.800000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000,-0.112500
11st,1.000000,-1.000000
1800 Contacts - Lens Store,0.838542,-0.300000
1LINE – One Line with One Touch,1.000000,-0.825000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,1.000000,-0.800000
21-Day Meditation Experience,0.587500,-0.265625
"2Date Dating App, Love and matching",1.000000,-0.645833
2GIS: directory & navigator,1.000000,-0.375000


In [42]:
end = time.time()
print ("##Il tempo di esecuzione del codice è stato di "+str(end-start)+" secondi##")

##Il tempo di esecuzione del codice è stato di 8.762563228607178 secondi##
